#SQL Lite


Install

    sudo apt-get install sqlite3

The SQLite libraries are supplied with an SQLite shell. Use this command to invoke the shell and create a database:

    sqlite3 mydatabase.db

Create a simple database that could be used for logging temperatures in different parts of a home. The first thing I need to do is create a table:


    BEGIN;
    CREATE TABLE temps (tdate DATE, ttime TIME, zone TEXT, temperature NUMERIC);
    COMMIT;

Next we need to add some data:

    BEGIN;
    INSERT INTO temps values(date('now', '-1 day'), time('now'), "kitchen", 20.6);
    INSERT INTO temps values(date('now', '-1 day'), time('now'), "greenhouse", 26.3);
    INSERT INTO temps values(date('now', '-1 day'), time('now'), "garage", 18.6);
    INSERT INTO temps values(date('now'), time('now', '-12 hours'), "kitchen", 19.5);
    INSERT INTO temps values(date('now'), time('now', '-12 hours'), "greenhouse", 15.1);
    INSERT INTO temps values(date('now'), time('now', '-12 hours'), "garage", 18.1);
    INSERT INTO temps values(date('now'), time('now'), "kitchen", 21.2);
    INSERT INTO temps values(date('now'), time('now'), "greenhouse", 27.1);
    INSERT INTO temps values(date('now'), time('now'), "garage", 19.1);
    COMMIT;
    
Now we can query the database using the SELECT command:

    SELECT * FROM temps WHERE zone="garage";
    
    SELECT * FROM temps
    
    SELECT temperature FROM temps WHERE zone="garage";
    
    SELECT * FROM temps WHERE tdate=date('now','-1 day');
    
    SELECT * FROM temps WHERE zone="kitchen" AND tdate=date('now','-1 day');
 

Using Python

    #!/usr/bin/env python

    import sqlite3

    conn=sqlite3.connect('mydatabase.db')

    curs=conn.cursor()

    print "\nEntire database contents:\n"
    for row in curs.execute("SELECT * FROM temps"):
        print row

    print "\nDatabase entries for the garage:\n"
    for row in curs.execute("SELECT * FROM temps WHERE zone='garage'"):
        print row

    conn.close()

Save this as database.py and make it executable with this command:

    chmod +x database.py

Each record is printed as a tuple of values, and text is in unicode. Each item in the tuple can be accessed using tuple notation (for example, to access the date from a row in the code above, you would use row[0]. You can convert from unicode using the Python str() function.

The example above simple returns database records in the order in they were found. Sometimes you might want to display the data in a particular order. This example selects all entries sorted in ascending order of temperature:

    print "\nDatabase entries in order of temperature\n"
    for row in curs.execute("SELECT * FROM temps ORDER BY temperature ASC"):
        print row

Suppose we plan on using a query repeatedly, but we might want to check for different thresholds. An alternative is to pass a variable to the execute function and let sqlite subsititute that variable in the query:

    print "\nDatabase entries where the temperature is above x:\n"
    for row in curs.execute("SELECT * FROM temps WHERE temperature>(?)",(x,)):
        print row
        
Now the query will return any record where the temperature is greater than x. Note that x has to be passed to the execute command as a tuple, not a single variable. In this case we're using a tuple with a single entry, and x is equal to 19.0. 

Using substitution is useful when writing code to insert a record into a database. In this function, the zone and temperature are passed to the query as variables, and the 'date' and 'time' SQL function is used to generate the current date and time:


    def add_temp_reading (zonestr, temp):
        # I used triple quotes so that I could break this string into
        # two lines for formatting purposes
        curs.execute("""INSERT INTO temperatures values(date('now'),
            time('now'), (?), (?))""", (zonestr,temp))

        # commit the changes
        conn.commit()

In [4]:
import sqlite3

# Note My sqlite3 database is located and running on raspberry pi 04, which is mapped to s on this computer
conn=sqlite3.connect('s:/mydatabase.db')

curs=conn.cursor()

print "\nEntire database contents:\n"
for row in curs.execute("SELECT * FROM temps"):
    print row

print "\nDatabase entries for the garage:\n"
for row in curs.execute("SELECT * FROM temps WHERE zone='garage'"):
    print row

conn.close()


Entire database contents:

(u'2015-11-13', u'19:49:50', u'kitchen', 20.6)
(u'2015-11-13', u'19:50:01', u'greenhouse', 26.3)
(u'2015-11-13', u'19:50:12', u'garage', 18.6)
(u'2015-11-14', u'07:50:27', u'kitchen', 19.5)
(u'2015-11-14', u'07:50:38', u'greenhouse', 15.1)
(u'2015-11-14', u'07:50:48', u'garage', 18.1)
(u'2015-11-14', u'19:51:07', u'kitchen', 21.2)
(u'2015-11-14', u'19:51:15', u'greenhouse', 27.1)
(u'2015-11-14', u'19:51:24', u'garage', 19.1)

Database entries for the garage:

(u'2015-11-13', u'19:50:12', u'garage', 18.6)
(u'2015-11-14', u'07:50:48', u'garage', 18.1)
(u'2015-11-14', u'19:51:24', u'garage', 19.1)


In [3]:
import sqlite3

conn=sqlite3.connect('s:/mydatabase.db')

curs=conn.cursor()

print "\nEntire database contents:\n"
for row in curs.execute("SELECT * FROM temps"):
    print row

print "\nDatabase entries for the garage:\n"
for row in curs.execute("SELECT * FROM temps WHERE zone='garage'"):
    print row

conn.close()


Entire database contents:

(u'2015-11-13', u'19:49:50', u'kitchen', 20.6)
(u'2015-11-13', u'19:50:01', u'greenhouse', 26.3)
(u'2015-11-13', u'19:50:12', u'garage', 18.6)
(u'2015-11-14', u'07:50:27', u'kitchen', 19.5)
(u'2015-11-14', u'07:50:38', u'greenhouse', 15.1)
(u'2015-11-14', u'07:50:48', u'garage', 18.1)
(u'2015-11-14', u'19:51:07', u'kitchen', 21.2)
(u'2015-11-14', u'19:51:15', u'greenhouse', 27.1)
(u'2015-11-14', u'19:51:24', u'garage', 19.1)

Database entries for the garage:

(u'2015-11-13', u'19:50:12', u'garage', 18.6)
(u'2015-11-14', u'07:50:48', u'garage', 18.1)
(u'2015-11-14', u'19:51:24', u'garage', 19.1)
